## Feature Extraction: CNN + Geometric & Temperature Descriptors

In [ ]:
# Path configuration for GitHub or local use
import os

# Main project folder
BASE_PATH = "./data/"

# Folder containing thermal images (CREATE THIS FOLDER MANUALLY)
# It is recommended to name the images based on the timestamp of the recording or the order in which the images are taken. 
# The format should follow HH-MM-SS (Hour-Minute-Second), e.g., 00-01-01.
IMAGE_PATH = os.path.join(BASE_PATH, "thermal_images/")

# --- FOLDERS AUTOMATICALLY CREATED IN BASE_PATH ---

# Folder for saving the CNN model
CNN_MODEL = os.path.join(BASE_PATH, "cnn_model/")

# Folder for saving CNN feature maps (activations)
ACTIVATION_PATH = os.path.join(BASE_PATH, "cnn_activations/")

# Folder for saving segment features: CNN cutouts, CSV with thermal and geometric features
RESULT_FEATURES = os.path.join(BASE_PATH, "result_features/")

# Ensure folders exist
for path in [BASE_PATH, ACTIVATION_PATH, IMAGE_PATH, RESULT_FEATURES, CNN_MODEL]:
    os.makedirs(path, exist_ok=True)


### 1. Build CNN Model for Multi-Scale Feature Extraction

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Flatten, concatenate
import keras
import os
import pickle

# Model configuration
activation = 'sigmoid'
optimizer = 'rmsprop'

# Input shape: height, width, channels
input_layer = Input(shape=(288, 382, 1))

# Parallel convolutional layers with different kernel sizes
conv1 = Conv2D(16, (2, 2), activation=activation, padding='same')(input_layer)
norm1 = BatchNormalization()(conv1)

conv2 = Conv2D(16, (3, 3), activation=activation, padding='same')(input_layer)
norm2 = BatchNormalization()(conv2)

conv3 = Conv2D(8, (5, 5), activation=activation, padding='same')(input_layer)
norm3 = BatchNormalization()(conv3)

conv4 = Conv2D(8, (7, 7), activation=activation, padding='same')(input_layer)
norm4 = BatchNormalization()(conv4)

# Concatenate and flatten the feature maps
concatenated = concatenate([norm1, norm2, norm3, norm4])
flat = Flatten()(concatenated)

# Define and compile the model
model = Model(inputs=input_layer, outputs=flat)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[keras.metrics.BinaryAccuracy()])

# Display model summary
print(model.summary())

# Save the model in TensorFlow .keras format
model_save_path = os.path.join(CNN_MODEL, "cnn_model.keras")

try:
    model.save(model_save_path)
    print(f"Model saved in TensorFlow format at: {model_save_path}")
except Exception as e:
    print(f"\nError while saving the model: {e}")


### 2. Process Input Images Using Pretrained CNN

In [ ]:
import os
import rioxarray
import numpy as np
from glob import glob
import warnings
import cv2
from tensorflow.keras.models import Model, load_model
import time
import gc

# Disable warnings
warnings.filterwarnings("ignore")
start_time = time.time()

model_path = os.path.join(CNN_MODEL, "cnn_model.keras")

# Load all TIFF images
image_files = glob(os.path.join(IMAGE_PATH, "*.tiff"))

# Image preprocessing function
def preprocess_image(file_path):
    img = rioxarray.open_rasterio(file_path)
    img = img[0].values
    img = cv2.resize(img, (382, 288))
    img = img.astype('float32')
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    img = np.expand_dims(img, axis=-1)
    return img

# Load pre-trained CNN model and define intermediate layer outputs
model = load_model(model_path)
activation_model = Model(inputs=model.input, outputs=[layer.output for layer in model.layers if 'conv' in layer.name])

# Initialize batching parameters
batch_size = 500
batch_counter = 0
image_counter = 0
all_activations = [[] for _ in range(len(activation_model.output))]

# Save mapping of processed images
mapping_file_path = os.path.join(ACTIVATION_PATH, 'image_mapping.txt')

with open(mapping_file_path, 'w') as f:
    f.write('img_path,img_name\n')

    # Process each image and extract activations
    for file in image_files:
        img = preprocess_image(file)
        img = np.expand_dims(img, axis=0)
        activations = activation_model.predict(img, verbose=0)

        # Save image metadata
        image_name = os.path.basename(file)
        f.write(f"{file},{image_name}\n")

        # Store layer activations
        for i, activation in enumerate(activations):
            all_activations[i].append(activation)

        image_counter += 1

        # Save batched activations
        if image_counter >= batch_size:
            for i, activations in enumerate(all_activations):
                activations = np.concatenate(activations, axis=0)
                activation_path = os.path.join(ACTIVATION_PATH, f"activations_layer_{i+1}_batch_{batch_counter+1}.npy")
                np.save(activation_path, activations)

            all_activations = [[] for _ in range(len(activation_model.output))]
            image_counter = 0
            batch_counter += 1
            gc.collect()

# Save remaining images (if any)
if image_counter > 0:
    for i, activations in enumerate(all_activations):
        activations = np.concatenate(activations, axis=0)
        activation_path = os.path.join(ACTIVATION_PATH, f"activations_layer_{i+1}_batch_{batch_counter+1}.npy")
        np.save(activation_path, activations)

end_time = time.time()
print(f"Execution time [min]: {(end_time - start_time)/60:.2f}")

print(f"Activations and image mapping saved successfully at: {ACTIVATION_PATH}")


### 3. Batch Merging for Feature Concatenation

In [ ]:
import numpy as np
import os
import gc
from glob import glob

# Loop through all desired activation layers
for nr in [1, 2, 3, 4]:
    print(f"\n--- Processing layer {nr} ---")

    # Find all batch files for the current layer
    layer_files = sorted(glob(os.path.join(ACTIVATION_PATH, f"activations_layer_{nr}_batch_*.npy")))
    if not layer_files:
        print(f"No batch files found for layer {nr}, skipping.")
        continue

    all_activations = []

    # Load and accumulate activations
    for file in layer_files:
        print(f"Loading {file}")
        activations_batch = np.load(file)
        all_activations.append(activations_batch)

    # Concatenate all batches into one array
    merged_activations = np.concatenate(all_activations, axis=0)

    # Save the merged activations
    output_file = os.path.join(ACTIVATION_PATH, f"activations_layer_{nr}.npy")
    np.save(output_file, merged_activations)
    print(f"Merged activations saved to: {output_file}")

    # Release memory and file handles
    del all_activations
    gc.collect()

    # Delete original batch files
    for file in layer_files:
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")


### 4. Segment Detection and Convolutional Feature Extraction

In [ ]:
import os
import numpy as np
import rioxarray as rxr
import cv2
from scipy.ndimage import gaussian_filter
import gc
import pandas as pd
from glob import glob
import json
import warnings
import time

gc.collect()
warnings.filterwarnings("ignore")

image_mapping_file = ACTIVATION_PATH + "image_mapping.txt"
mapping_csv_file = RESULT_FEATURES + "image_features_mapping.csv"

# Utility functions
def contour_to_string(contour):
    return json.dumps(contour.squeeze().tolist())

def adjust_window_to_bounds(cx, cy, image_shape, window_size=30, min_margin=3):
    x_start = cx - window_size // 2
    y_start = cy - window_size // 2
    x_end = x_start + window_size
    y_end = y_start + window_size

    if (x_start < min_margin or x_end > image_shape[1] - min_margin or
        y_start < min_margin or y_end > image_shape[0] - min_margin):
        return None
    return x_start, y_start, x_end, y_end

# Load image mapping
image_mapping = {}
with open(image_mapping_file, 'r') as f:
    for line in f.readlines()[1:]:
        img_path, img_name = line.strip().split(',')
        image_mapping[img_name] = img_path

# Load activation files
activation_files = glob(os.path.join(ACTIVATION_PATH, "activations_layer_*.npy"))

# Parameters
min_area = 1
max_area = 50
window_size = 30
# Minimum RMS deviation (from median temperature) in DN required to accept a detected segment
rms_treshold = 100 # thermal resolution of 100 DN per 1°C in 16-bit thermal image

data_for_csv = []
is_csv_saved = False

start_time = time.time()

# Process each activation file (corresponding to a CNN layer)
for activation_file in activation_files:
    activations = np.load(activation_file, mmap_mode='r')
    num_filters = activations.shape[-1]
    layer_name = os.path.basename(activation_file).replace('.npy', '')
    all_cutouts_for_layer = []

    # Process each image in the current activation file
    for img_idx in range(activations.shape[0]):
        activation_map = activations[img_idx]

        img_name = list(image_mapping.keys())[img_idx]
        img_path = os.path.join(IMAGE_PATH, image_mapping[img_name])
        original_image = rxr.open_rasterio(img_path).squeeze()
        
        # Normalize and smooth the image for thresholding
        X = np.array(original_image, dtype=np.int16)
        X_normalized = cv2.normalize(X, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        X_smoothed = gaussian_filter(X_normalized, sigma=0.95)

        adaptive_thresh = cv2.adaptiveThreshold(X_smoothed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                cv2.THRESH_BINARY, 13, -8)
        contours, _ = cv2.findContours(adaptive_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours by area and select top 10 by temperature
        filtered_contours = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if min_area < area < max_area:
                mask_contour = np.zeros_like(X, dtype=np.int16)
                cv2.drawContours(mask_contour, [cnt], -1, 1, thickness=cv2.FILLED)
                masked_temps = np.ma.masked_array(X, mask=~(mask_contour.astype(bool)))
                max_temp = masked_temps.max()
                filtered_contours.append((cnt, max_temp, area))
        filtered_contours = sorted(filtered_contours, key=lambda x: x[1], reverse=True)

        coordinates = []

        # Extract windowed cutouts from activation map
        for cnt, max_temp, original_area in filtered_contours:
            M = cv2.moments(cnt)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                cnt_str = contour_to_string(cnt)

                result = adjust_window_to_bounds(cx, cy, activation_map.shape, window_size, min_margin=4)
                if result is not None:
                    x_start, y_start, x_end, y_end = result

                    # Compute root mean square of temperature deviations from the median (robust measure of variability)
                    mask_contour = np.zeros_like(X, dtype=np.uint16)
                    cv2.drawContours(mask_contour, [cnt], -1, 255, thickness=cv2.FILLED)
                    masked_temps = np.ma.masked_array(X, mask=~(mask_contour.astype(bool)))

                    median_temp = np.ma.median(masked_temps)
                    std_temp = masked_temps.std()
                    rms_from_median = np.sqrt(np.mean((masked_temps - median_temp) ** 2))

                    if rms_from_median >= rms_treshold:
                        cropped_image = activation_map[y_start:y_end, x_start:x_end, :]
                        all_cutouts_for_layer.append(cropped_image)
                        coordinates.append([img_path, img_name, cx, cy, cnt_str])
                else:
                    continue

        if not is_csv_saved:
            data_for_csv.extend(coordinates)

    # Save cropped cutouts for the current layer
    if all_cutouts_for_layer:
        all_cutouts_array = np.array(all_cutouts_for_layer)
        cutout_save_path = os.path.join(RESULT_FEATURES, f"{layer_name}_cutouts.npy")
        np.save(cutout_save_path, all_cutouts_array)
        print(f"{layer_name} - shape: {all_cutouts_array.shape}")

    # Save CSV with contour mapping
    if not is_csv_saved:
        df = pd.DataFrame(data_for_csv, columns=['img_path', 'img_name', 'cx', 'cy', 'contour'])
        df.to_csv(mapping_csv_file, index=False)
        is_csv_saved = True

    gc.collect()

end_time = time.time()
print(f"Execution time [min]: {(end_time - start_time)/60:.2f}")

print(f"Saved to: {RESULT_FEATURES}")


### 5. Geometric and Temperature Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import rioxarray
import cv2
import os
import json
import warnings
import time

warnings.filterwarnings("ignore")

# Paths
mapping_csv_file = RESULT_FEATURES + "image_features_mapping.csv"

# Utility functions for contour serialization
def string_to_contour(contour_string):
    return np.array(json.loads(contour_string), dtype=np.int32).reshape(-1, 1, 2)

def contour_to_string(contour):
    return json.dumps(contour.squeeze().tolist())

# Fit PCA to extract ellipse parameters
def fit_ellipse_pca(points, scale_factor=1.0):
    pca = PCA(n_components=2)
    pca.fit(points)
    center = pca.mean_
    axes = np.sqrt(pca.explained_variance_) * 2 * scale_factor
    angle = np.degrees(np.arctan2(pca.components_[0, 0], pca.components_[0, 1]))
    projections = pca.transform(points)
    return center, axes, angle, projections

# Generate points within a specified radius
def generate_points_in_radius(center, radius):
    points_in_radius = []
    cy, cx = center
    for y in range(int(cy - radius), int(cy + radius) + 1):
        for x in range(int(cx - radius), int(cx + radius) + 1):
            if np.sqrt((x - cx)**2 + (y - cy)**2) <= radius:
                points_in_radius.append((y, x))
    return np.array(points_in_radius)
    
# Check if a point lies inside an ellipse
def is_point_in_ellipse(y, x, center, axes, angle, margin=0.2):
    cy, cx = center
    a, b = axes[0], axes[1]
    angle_rad = np.radians(angle)
    cos_angle = np.cos(angle_rad)
    sin_angle = np.sin(angle_rad)
    x_shifted = x - cx
    y_shifted = y - cy
    x_rot = cos_angle * x_shifted + sin_angle * y_shifted
    y_rot = -sin_angle * x_shifted + cos_angle * y_shifted
    return (x_rot**2) / (a**2) + (y_rot**2) / (b**2) <= 1 + margin

# Calculate ellipse circumference (Ramanujan approximation)
def calculate_ellipse_circumference(axes):
    a, b = axes
    return np.pi * (3*(a + b) - np.sqrt((3*a + b)*(a + 3*b)))

# Calculate ellipse area
def calculate_ellipse_area(axes):
    a, b = axes
    return np.pi * a * b

# Main feature extraction loop
def main_process(mapping_csv_file):
    df = pd.read_csv(mapping_csv_file)
    df['contour'] = df['contour'].apply(string_to_contour)
    neighborhood_features = []

    for index, row in df.iterrows():
        image_path = row['img_path']
        img_name = row['img_name']
        contour = row['contour']
        cx = row['cx']
        cy = row['cy']

        image = rioxarray.open_rasterio(image_path)
        X = np.array(image[0], dtype=np.int16)

        # Create binary mask from contour
        mask_contour = np.zeros_like(X, dtype=np.uint16)
        cv2.drawContours(mask_contour, [contour], -1, 255, thickness=cv2.FILLED)
        points = np.column_stack(np.where(mask_contour == 255))

        if len(points) > 3:
            center, axes, angle, projections = fit_ellipse_pca(points)

            # Define elliptical neighborhood
            radius = 15 # half of the window size in pixels  
            points_in_radius = generate_points_in_radius(center, radius)
            inside_ellipse_mask = np.array([is_point_in_ellipse(y, x, center, axes, angle) for (y, x) in points_in_radius])
            points_inside_actual_ellipse = points_in_radius[inside_ellipse_mask]
            points_inside_actual_ellipse = np.clip(points_inside_actual_ellipse, 0, np.array(X.shape) - 1)

            # Define scaled ellipse (150% size)
            scaled_axes = axes * 1.5
            inside_scaled_ellipse_mask = np.array([is_point_in_ellipse(y, x, center, scaled_axes, angle) for (y, x) in points_in_radius])
            points_remaining = points_in_radius[inside_scaled_ellipse_mask & ~inside_ellipse_mask]
            points_remaining = np.clip(points_remaining, 0, np.array(X.shape) - 1)

            # Temperature statistics
            temperatures_inside_ellipse = X[points_inside_actual_ellipse[:, 0], points_inside_actual_ellipse[:, 1]]
            temperatures_remaining = X[points_remaining[:, 0], points_remaining[:, 1]]

            max_temp_ellipse = np.max(temperatures_inside_ellipse)
            min_temp_ellipse = np.min(temperatures_inside_ellipse)
            max_temp_remaining = np.max(temperatures_remaining)
            min_temp_remaining = np.min(temperatures_remaining)

            max_temp_diff = max_temp_ellipse - max_temp_remaining
            min_temp_diff = min_temp_ellipse - min_temp_remaining
            remaining_temp_range = max_temp_remaining - min_temp_remaining
            ellipse_temp_range = max_temp_ellipse - min_temp_ellipse

            # PCA and geometric features
            std_pca_component_1 = projections[:, 0].std()
            std_pca_component_2 = projections[:, 1].std()
            eccentricity = np.sqrt(1 - (axes[1] / axes[0])**2)
            circumference = calculate_ellipse_circumference(axes)
            area = calculate_ellipse_area(axes)

            temperatures_points = X[points[:, 0], points[:, 1]]
            median_temp_points = np.median(temperatures_points)
            median_temp_remaining_points = np.median(temperatures_remaining)
            difference_in_medians = median_temp_points - median_temp_remaining_points

            cnt_str = contour_to_string(contour)

            neighborhood_features.append({
                'img_path': image_path,
                'img_name': img_name,
                'cx': cx,
                'cy': cy,
                'Major Axes': axes[0],
                'Minor Axes': axes[1],
                'Std PCA 1': std_pca_component_1,
                'Std PCA 2': std_pca_component_2,
                'Eccentricity': eccentricity,
                'Circumference': circumference,
                'Area': area,
                'Median Difference': difference_in_medians,
                'Max Difference': max_temp_diff,
                'Min Difference': min_temp_diff,
                'Remaining Temp Range': remaining_temp_range,
                'Ellipse Temp Range': ellipse_temp_range
            })

        else:
            print(f"Too few points to fit ellipse for image: {image_path}")

    df = pd.DataFrame(neighborhood_features)
    return df
    
start_time = time.time()

# Run the process and generate output CSV
df = main_process(mapping_csv_file)

# Add segment index column
df.insert(0, 'idx', range(len(df)))

# Save to CSV
csv_name = 'image_features_elipse.csv'
features_df = os.path.join(RESULT_FEATURES, csv_name)
df.to_csv(features_df, sep=';', index=False)

end_time = time.time()
print(f"Execution time [min]: {(end_time - start_time)/60:.2f}")

print(f"Saved to: {RESULT_FEATURES}")

df


### 6. Feature Mapping to Structured Format

In [ ]:
import numpy as np
import pandas as pd

csv_file_path = RESULT_FEATURES + 'image_features_elipse.csv'
feature_mapping = RESULT_FEATURES + 'feature_mapping.txt'

# Load activation feature arrays
activations_layer_1 = np.load(RESULT_FEATURES + 'activations_layer_1_cutouts.npy')
activations_layer_2 = np.load(RESULT_FEATURES + 'activations_layer_2_cutouts.npy')
activations_layer_3 = np.load(RESULT_FEATURES + 'activations_layer_3_cutouts.npy')
activations_layer_4 = np.load(RESULT_FEATURES + 'activations_layer_4_cutouts.npy')

# Generate descriptive names for CNN activation features
def generate_filter_names():
    filter_names = []
    filters_per_layer = {
        1: activations_layer_1.shape[-1],
        2: activations_layer_2.shape[-1],
        3: activations_layer_3.shape[-1],
        4: activations_layer_4.shape[-1]
    }
    for layer in range(1, 5):
        num_filters = filters_per_layer[layer]
        for filter_idx in range(num_filters):
            filter_names.append(f"Activation {layer} filter {filter_idx}")
    return filter_names

# Generate names for CNN features
npy_feature_names = generate_filter_names()

# Load CSV feature file
csv_data = pd.read_csv(csv_file_path, sep=';')

# Extract all handcrafted (non-CNN) feature names from column index 5 onward
csv_feature_names = csv_data.columns[5:].tolist()

# Combine CNN and handcrafted feature names
all_feature_names = npy_feature_names + csv_feature_names

# Display feature names for verification
print("CSV feature names:")
print(', '.join(f"'{name}'" for name in csv_feature_names))

print("\nTotal features (CNN + CSV):", len(all_feature_names))

# Save all feature names to a text file
with open(feature_mapping, 'w') as f:
    for feature_name in all_feature_names:
        f.write(f"{feature_name}\n")

print(f"\nFeature mapping saved to {RESULT_FEATURES}")


### 7. Visualize and Label Segments (Dataset/Class Columns)

In [ ]:
import pandas as pd
import rioxarray as rxr
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Rectangle
import warnings

warnings.filterwarnings("ignore")

features_csv_path = os.path.join(RESULT_FEATURES, "image_features_elipse.csv")

# Window size for drawing rectangles around detected segments
window_size = 30

# Load features DataFrame with 'idx' column
df = pd.read_csv(features_csv_path, sep=';')

# Function to visualize segments on the original image
def visualize_segments(img_name, central_points, idx_labels):
    img_path = os.path.join(IMAGE_PATH, img_name)
    
    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        return
    
    original_image = rxr.open_rasterio(img_path).squeeze()
    original_image_np = original_image.values

    plt.figure(figsize=(5, 5))
    plt.imshow(original_image_np, cmap='magma')

    for (cx, cy), idx in zip(central_points, idx_labels):
        rect = Rectangle((cx - window_size // 2, cy - window_size // 2), 
                         window_size, window_size, linewidth=1, 
                         edgecolor='red', facecolor='none')
        plt.gca().add_patch(rect)
        plt.annotate(f"{idx}", (cx - window_size // 2 + 2, cy - window_size // 2 - 1), 
                     color='white', weight='bold', fontsize=6, bbox=dict(boxstyle="round,pad=0.2", facecolor="black", edgecolor="none", alpha=0.4))

    plt.title(f"Segments on image: {os.path.basename(img_name)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Group data by image and visualize segments for each image
for img_name, group in df.groupby('img_name'):
    central_points = list(zip(group['cx'], group['cy']))
    idx_labels = group['idx'].tolist()
    
    visualize_segments(img_name, central_points, idx_labels)
